In [106]:
#!/usr/bin/env python

import numpy as np
import cv2
from numpy.core.numeric import full
import rospy
import rospkg
import open3d as o3d
from transforms3d.quaternions import mat2quat
import time
import copy
import os
from copy import deepcopy

import sys
from os import path
from ocrtoc_common.camera_interface import CameraInterface
from ocrtoc_common.transform_interface import TransformInterface
from sensor_msgs.msg import CameraInfo, Image, PointCloud2
import pickle


if __name__ == "__main__" :
    rospack = rospkg.RosPack()
    taskid= '2-1-2-data-2'
    save_path = os.path.join(rospack.get_path('ocrtoc_perception'),'data',taskid, '')
    full_pcd = o3d.io.read_point_cloud(save_path + 'full_pcd.pcd') 
    with open(save_path +  'color_images.pickle', 'rb') as handle:
        color_images = pickle.load(handle)  
    with open(save_path + 'camera_poses.pickle', 'rb') as handle:
        camera_poses = pickle.load(handle) 
    # Compute Grasping Poses (Many Poses in a Scene)
    with open(save_path + 'gg.pickle', 'rb') as handle:
        gg = pickle.load(handle)      

    with open(save_path + 'object_poses.pickle', 'rb') as handle:
        object_poses= pickle.load(handle) 
        
        
        
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(0.1)        


[Open3D WARNING] Read PCD failed: unable to open file: /root/ocrtoc_ws/src/ocrtoc_perception/data/2-1-2-data-2/full_pcd.pcd


FileNotFoundError: [Errno 2] No such file or directory: '/root/ocrtoc_ws/src/ocrtoc_perception/data/2-1-2-data-2/color_images.pickle'

In [51]:
#for object_ in object_poses.values():
    #print(object_["pose"])
    
object_pose_list = list(object_poses.values())
object_pose_list_len = len(object_pose_list)

object_origin_extracted = [None]*object_pose_list_len

for i in range(object_pose_list_len):
    object_pose = object_pose_list[i]['pose']
    #object_rot = mat2quat(object_pose[0:3,0:3])
    object_trans = object_pose[0:3,3]
    
    #p_stamped.pose.position.x = object_trans[0]
    #p_stamped.pose.position.y = object_trans[1]
    #p_stamped.pose.position.z = object_trans[2]

    #p_stamped.pose.orientation.x = object_rot[0]
    #p_stamped.pose.orientation.y = object_rot[1]
    #p_stamped.pose.orientation.z = object_rot[2]
    #p_stamped.pose.orientation.w = object_rot[3]

    #p_stamped.header.frame_id = 'world'

    object_origin_extracted[i] = object_trans
    
    
print(object_origin_extracted)

[array([ 0.11676473, -0.18633811,  0.03117309]), array([ 0.11507295, -0.21543022,  0.06179549]), array([-0.07621131, -0.14495013,  0.04011803]), array([-0.07736078,  0.11978251,  0.05341744]), array([-0.05545778,  0.14673042,  0.02042893])]


In [70]:
print(gg.scores)
print(object_poses)


def assign_grasps_to_object(ts, object_poses):
    dist_thresh = np.inf
    min_dists = np.inf * np.ones((len(rs)))
    min_object_ids = -1 * np.ones(shape = (len(rs)), dtype = np.int32)
    for i, object_name in enumerate(object_poses.keys()):
        #object_names.append(object_name)
        object_pose = object_poses[object_name]

        dists = np.linalg.norm(ts - object_pose['pose'][:3,3], axis=1)
        object_mask = np.logical_and(dists < min_dists, dists < dist_thresh)

        min_object_ids[object_mask] = i
        min_dists[object_mask] = dists[object_mask]
    return object_mask, min_object_ids


def center_of_mass_score(gg, object_poses):
    penalty_factor = 1
    ts = gg.translations
    rs = gg.rotation_matrices
    depths = gg.depths
    ts = ts + rs[:,:,0]*(np.vstack((depths, depths, depths)).T)
    dists = []
    object_mask = assign_graps_to_object(object_poses)
    for object_ in object_poses:
        #if object_
        dists.append(np.linalg.norm(ts-object_['pose'][:3,3], axis=1))
        
    gg.scores = gg.scores - dists*penalty_factor
    return gg


#gg = center_of_mass_score(gg, object_poses)
#print(gg.scores)

[0.35777852 0.12361464 0.43927532 0.12490012 0.23966826 0.5194343
 0.8115901  0.14152083 0.28402114 0.55195725 1.2477342  0.2191413
 0.39679337 0.40376797 0.2248253  0.25346917 1.0360943  0.30572402
 0.3645935  0.2051539  0.81941813 0.19512825 1.1890011  0.98528194
 1.0278609  1.5065768  0.3994226  0.947345   0.08543259 0.730855
 1.3978658  0.5912513  0.5603831  0.12164972 1.0902991  0.3777262
 0.46180457 0.9501651  0.16380483 0.99963075 1.0652131  1.1941316
 0.6904812  0.81127197 0.28882182 0.26138377 0.15658262 0.86507297
 0.39867026 0.24011266 0.5211815  0.1511082  0.41391677 0.2418275
 0.33916417 0.7252431  0.911852   0.13555767 0.74057937 0.49974072
 0.8521794 ]
{'suger_1': {'pose': array([[ 0.99776771, -0.01114179,  0.06584419,  0.11676473],
       [ 0.01355452,  0.99924862, -0.03631063, -0.18633811],
       [-0.06539015,  0.03712206,  0.99716903,  0.03117309],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])}, 'suger_2': {'pose': array([[ 0.61729883, -0.78604833, -0

In [71]:
ts = gg.translations
rs = gg.rotation_matrices
depths = gg.depths
ts = ts + rs[:,:,0]*(np.vstack((depths, depths, depths)).T)
assign_grasps_to_object(ts, object_poses)


(array([ True, False,  True,  True,  True, False, False,  True,  True,
         True, False,  True,  True,  True,  True, False, False,  True,
         True, False, False,  True, False, False, False,  True, False,
        False,  True,  True, False, False,  True,  True, False,  True,
         True, False, False,  True,  True, False, False, False,  True,
         True,  True, False,  True,  True,  True,  True,  True,  True,
        False, False, False,  True, False, False, False]),
 array([4, 3, 4, 4, 4, 3, 0, 4, 4, 4, 2, 4, 4, 4, 4, 3, 0, 4, 4, 3, 2, 4,
        0, 2, 2, 4, 3, 0, 4, 4, 2, 3, 4, 4, 0, 4, 4, 0, 3, 4, 4, 0, 2, 0,
        4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 3, 0, 2, 4, 0, 3, 0], dtype=int32))

In [52]:
print(gg.translations)

[[ 0.0231152   0.25075957  0.13020846]
 [-0.1790488   0.26174837  0.13261144]
 [ 0.09479912  0.10873193  0.07119028]
 [ 0.14260983  0.25382715  0.08035313]
 [-0.16391094  0.25354818  0.04794811]
 [-0.1450553   0.14065886  0.0505309 ]
 [ 0.10697125 -0.21362303  0.05687856]
 [ 0.09330377  0.25688225  0.1301308 ]
 [-0.03551773  0.25100017  0.13018793]
 [-0.12838559  0.25255713  0.1297549 ]
 [-0.12885766 -0.1491029   0.03138361]
 [ 0.16923319  0.25338054  0.04489009]
 [ 0.11995343  0.13206185  0.06871444]
 [ 0.08559955  0.14207825  0.07010586]
 [-0.08667577  0.25445232  0.12832566]
 [-0.17158613  0.24982502  0.09130426]
 [ 0.11829621 -0.16924958  0.02778559]
 [ 0.05975473  0.2569569   0.11405578]
 [ 0.1744692   0.25308692  0.0852924 ]
 [-0.1478417   0.25462055  0.11920601]
 [-0.09622861 -0.14656672  0.07822736]
 [ 0.08086799  0.12174106  0.05172231]
 [ 0.09703867 -0.19905841  0.03673489]
 [-0.02894979 -0.14808685  0.06087001]
 [-0.13041648 -0.1493197   0.06091681]
 [ 0.11582229  0.1223314 

In [ ]:
o3d.visualization.draw_geometries([frame, full_pcd, *gg.to_open3d_geometry_list()])


In [95]:
np.arg(gg.scores)

6

In [81]:
print(full_pcd.segment_plane)

dir(full_pcd)

<bound method PyCapsule.segment_plane of PointCloud with 774972 points.>


['HalfEdgeTriangleMesh',
 'Image',
 'LineSet',
 'PointCloud',
 'RGBDImage',
 'TetraMesh',
 'TriangleMesh',
 'Type',
 'Unspecified',
 'VoxelGrid',
 '__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'cluster_dbscan',
 'colors',
 'compute_convex_hull',
 'compute_mahalanobis_distance',
 'compute_mean_and_covariance',
 'compute_nearest_neighbor_distance',
 'compute_point_cloud_distance',
 'create_from_depth_image',
 'create_from_rgbd_image',
 'crop',
 'dimension',
 'estimate_normals',
 'get_axis_aligned_bounding_box',
 'get_center',
 'get_geometry_type',
 'get_max_bound',
 'get_min_bound',
 'get_oriented_bounding_box',
 'get_rotation_matrix_